<a href="https://colab.research.google.com/github/aradhana148/Medical-NER-RE/blob/master/radgraph_ct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q \
  torch>=2.1.0 \
  transformers>=4.39.0 \
  appdirs \
  jsonpickle \
  filelock \
  h5py \
  nltk \
  dotmap \
  pytest


In [2]:
!pip install radgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.0/588.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for radgraph: filename=radgraph-0.1.18-py3-none-any.whl size=812635 sha256=1b80bf0caf1c8963efae132fa373db78709bb4ede0d1af09bd4d74ea85698d78
  Stored in directory: /root/.cache/pip/wheels/fb/3c/fb/214f5d5cdab2a0f9f0904fd81d7fd1134404100b4444554df8
Successfully built radgraph


In [3]:
import json
from radgraph import get_radgraph_processed_annotations, RadGraph

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
import pandas as pd
import ast
import re


In [5]:
# def clean_text(x):
#     if not x:
#         return ""
#     if isinstance(x, list):
#         return ", ".join(x)
#     return str(x)


# def annotation_to_sentence(annotation):
#     # obs = clean_text(annotation.get("observation"))
#     # loc = clean_text(annotation.get("located_at"))
#     # sug = clean_text(annotation.get("suggestive_of"))
#     # tag = annotation.get("tags", [""])[0]

#     # # Normalize tag
#     # tag = re.sub("_", " ", tag)

#     # parts = []

#     # # Handle ABSENT case first
#     # if tag == "definitely absent":
#     #     if obs:
#     #         sentence = f"No {obs}"
#     #         if loc:
#     #             sentence += f" in the {loc}"
#     #         sentence += "."
#     #         return sentence

#     # # PRESENT (or unknown) case
#     # if obs:
#     #     parts.append(obs.capitalize())

#     # if loc:
#     #     parts.append(f"in the {loc}")

#     # if sug:
#     #     parts.append(f"suggestive of {sug}")

#     # sentence = " ".join(parts).strip()

#     # if sentence and not sentence.endswith("."):
#     #     sentence += "."

#     # return sentence
#     obs = clean_text(annotation.get("observation"))
#     loc = clean_text(annotation.get("located_at"))
#     sug = clean_text(annotation.get("suggestive_of"))
#     tag = annotation.get("tags", [""])[0]

#     # Normalize tag
#     tag = re.sub("_", " ", tag)

#     modified_obs = obs # Initialize modified_obs

#     parts = []

#     # Handle ABSENT case first
#     if tag == "definitely absent":
#         if obs:
#             sentence = f"No {obs}"
#             if loc:
#                 sentence += f" in the {loc}"
#             sentence += "."
#             return sentence
#     # Handle other descriptive tags
#     elif tag.startswith("definitely ") and tag not in ["definitely absent", "definitely present"]:
#         descriptive_tag = tag.replace("definitely ", "").strip()
#         if modified_obs:
#             modified_obs = f"{descriptive_tag} {modified_obs}"
#         else:
#             modified_obs = descriptive_tag
#     elif tag and tag not in ["definitely present", "definitely absent", "present", "absent", "uncertain", "likely", "possibly"]:
#         if modified_obs:
#             modified_obs = f"{tag} {modified_obs}"
#         else:
#             modified_obs = tag

#     # PRESENT (or unknown) case
#     if modified_obs:
#         parts.append(modified_obs.capitalize())

#     if loc:
#         parts.append(f"in the {loc}")

#     if sug:
#         parts.append(f"suggestive of {sug}")

#     sentence = " ".join(parts).strip()

#     if sentence and not sentence.endswith("."):
#         sentence += "."

#     return sentence

In [109]:
# def clean_text(x):
#     if not x:
#         return ""
#     if isinstance(x, list):
#         return ", ".join(x)
#     return str(x)


# def annotation_to_sentence(annotation):
#     obs = clean_text(annotation.get("observation"))
#     loc = clean_text(annotation.get("located_at"))
#     sug = clean_text(annotation.get("suggestive_of"))
#     tag = annotation.get("tags", [""])[0]

#     # Normalize tag
#     tag = re.sub("_", " ", tag)

#     modified_obs = obs # Initialize modified_obs

#     parts = []

#     # Handle ABSENT case first
#     if tag == "definitely absent":
#         if obs:
#             # Prefer "No [loc] [obs]" over "No [obs] in the [loc]"
#             if loc:
#                 sentence = f"No {loc} {obs}"
#             else:
#                 sentence = f"No {obs}"
#             sentence += "."
#             return sentence.capitalize()
#         elif loc: # If no observation but there's a location, assume "no abnormality"
#             return f"No abnormality in the {loc}.".capitalize()
#         else:
#             return "" # Cannot form a coherent sentence for "absent" without observation or location

#     # --- Handle PRESENT/DESCRIPTIVE tags ---
#     # Handle other descriptive tags that directly modify the observation
#     if tag.startswith("definitely ") and tag not in ["definitely absent", "definitely present"]:
#         descriptive_tag = tag.replace("definitely ", "").strip()
#         if modified_obs:
#             modified_obs = f"{descriptive_tag} {modified_obs}"
#         else:
#             # If original obs was empty, add "findings" to the descriptive tag
#             modified_obs = f"{descriptive_tag} findings"
#     elif tag and tag not in ["definitely present", "definitely absent", "present", "absent", "uncertain", "likely", "possibly"]:
#         if modified_obs:
#             modified_obs = f"{tag} {modified_obs}"
#         else:
#             # If original obs was empty, add "findings" to the descriptive tag
#             modified_obs = f"{tag} findings"

#     # Combine suggestive_of with observation
#     if modified_obs and sug:
#         modified_obs = f"{modified_obs} suggestive of {sug}"
#     elif not modified_obs and sug: # If modified_obs is empty and sug is present
#         modified_obs = f"findings suggestive of {sug}"

#     # PRESENT (or unknown) case
#     if modified_obs:
#         parts.append(modified_obs.capitalize())
#         if loc:
#             parts.append(f"in the {loc}") # Append location only if observation/suggestion is present

#     sentence = " ".join(parts).strip()

#     if sentence and not sentence.endswith("."):
#         sentence += "."

#     return sentence

In [46]:
def clean_text(x):
    if not x:
        return ""
    if isinstance(x, list):
        return ", ".join(x)
    return str(x)


def annotation_to_sentence(annotation):
    obs = clean_text(annotation.get("observation"))
    loc = clean_text(annotation.get("located_at"))
    sug = clean_text(annotation.get("suggestive_of"))
    tag = annotation.get("tags", [""])[0]

    # Normalize tag
    tag = re.sub("_", " ", tag)

    modified_obs = obs # Initialize modified_obs

    parts = []

    # Handle ABSENT case first
    if tag == "definitely absent":
        if obs:
            # As per instruction: 'No {observation}' or 'No {observation} in the {location}'
            sentence = f"No {obs}"
            if loc:
                sentence += f" in the {loc}"
            sentence += "."
            return sentence.capitalize()
        elif loc: # If no observation but there's a location, assume "no abnormality"
            return f"No abnormality in the {loc}.".capitalize()
        else:
            return "" # Cannot form a coherent sentence for "absent" without observation or location

    # --- Handle PRESENT/DESCRIPTIVE tags ---
    # Handle other descriptive tags that directly modify the observation
    if tag.startswith("definitely ") and tag not in ["definitely absent", "definitely present"]:
        descriptive_tag = tag.replace("definitely ", "").strip()
        if modified_obs:
            modified_obs = f"{descriptive_tag} {modified_obs}"
        else:
            # If original obs was empty, add "findings" to the descriptive tag
            modified_obs = f"{descriptive_tag} findings"
    elif tag and tag not in ["definitely present", "definitely absent", "present", "absent", "uncertain", "likely", "possibly"]:
        if modified_obs:
            modified_obs = f"{tag} {modified_obs}"
        else:
            # If original obs was empty, add "findings" to the descriptive tag
            modified_obs = f"{tag} findings"

    # Combine suggestive_of with observation
    if modified_obs and sug:
        modified_obs = f"{modified_obs} suggestive of {sug}"
    elif not modified_obs and sug: # If modified_obs is empty and sug is present
        modified_obs = f"findings suggestive of {sug}"

    # PRESENT (or unknown) case
    if modified_obs:
        parts.append(modified_obs.capitalize())
        if loc:
            parts.append(f"in the {loc}") # Append location only if observation/suggestion is present

    sentence = " ".join(parts).strip()

    if sentence and not sentence.endswith("."):
        sentence += "."

    return sentence

In [47]:

df = pd.read_csv("/content/ct_data.csv")

clean = (
    df.iloc[9:19, 0]
      .astype(str)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
      .reset_index(drop=True)
)

concepts = df.iloc[0:10, 1].reset_index(drop=True)

eval_df = pd.DataFrame({"report": clean, "concepts": concepts})
model_type = "modern-radgraph-xl"
radgraph = RadGraph(model_type=model_type)
reports = eval_df["report"].astype(str).tolist()

pred_concepts = []

for report in reports:
    annotations = radgraph([report])  # one report batch
    processed = get_radgraph_processed_annotations(annotations)  # dict with "processed_annotations"

    sents = []
    for ann in processed["processed_annotations"]:
        s = annotation_to_sentence(ann)
        if s:
            sents.append(s)

    pred_concepts.append(list(dict.fromkeys(sents)))

reports = eval_df["report"].astype(str).tolist()

gt_concepts = [
    ast.literal_eval(x) if pd.notna(x) else []
    for x in eval_df["concepts"].tolist()
]


Using device: cpu


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [48]:
for i in range(len(pred_concepts)):
  print(i)
  print(reports[i])
  print(gt_concepts[i])
  print(pred_concepts[i])

0
The trachea and both main bronchi are open. An air cyst is observed at the anterobasal level of the right lower lobe. Focal consolidative density is observed at the paramediastinal level of the right middle lobe. There are focal scarring changes at the posterobasal level of the left lung. A calcified nodule measuring 3 mm is observed at the laterobasal level. No bilateral pleural effusion or pneumothorax is detected. Mediastinal main vascular structures, heart contour, and size are normal. Thoracic aorta diameter is normal. No pericardial effusion or thickening is observed. Thoracic esophagus is normal in caliber without significant wall thickening. No enlarged lymph nodes are detected in prevascular, pre-paratracheal, subcarinal, or bilateral hilar-axillary regions. Bone structures within the field of view are normal. Vertebral body heights are preserved. A nodular formation compatible with an accessory spleen is observed adjacent to the spleen.
['patent air passages', 'nodular cons

In [49]:
# def canonicalize_concept(s: str) -> list[str]:
#     s = str(s).strip().lower()
#     s = re.sub(r"\s+", " ", s)
#     s = s.rstrip(".")

#     # Split concepts by comma for adjective propagation - now applied uniformly
#     split_concepts = [c.strip() for c in s.split(',')]
#     processed_concepts = []

#     for concept_str in split_concepts:
#         # Rule 1: Propagate leading adjectives (e.g., 'normal') across comma-separated concepts
#         # Apply if there's more than one part after splitting by comma
#         if len(split_concepts) > 1: # Removed 'is_pred' condition
#             # Get the leading adjective from the *first* part of the comma-separated string
#             first_part_raw = split_concepts[0]
#             match_leading_adj = re.match(r"^(\w+)", first_part_raw)
#             leading_adjective = match_leading_adj.group(1) if match_leading_adj else ""

#             # Apply adjective to current concept_str if it doesn't already start with an adjective or "no"
#             current_concept_first_word = concept_str.split(' ')[0]
#             if leading_adjective and \
#                current_concept_first_word not in ["normal", "mild", "moderate", "severe", "slight", "minimal", "increased", "decreased", "diffuse", "bilateral", "patchy", "clear", "enlarged", "stable", "unchanged", "intact", "no"] and \
#                concept_str != first_part_raw: # Ensure we don't re-add to the first part
#                 concept_str = f"{leading_adjective} {concept_str}"

#         # Canonicalization rules from original function
#         concept_str = concept_str.replace("top - normal", "normal")
#         concept_str = concept_str.replace("within normal limits", "normal")
#         concept_str = concept_str.replace("unremarkable", "normal")

#         # reorder templates to match GT phrase style
#         # "no effusion in the pleural" -> "no pleural effusion"
#         # Modified regex to handle both 'no' and 'normal' as leading modifiers for reordering
#         m = re.match(r"^(no|normal) (.+?) in the (.+)$", concept_str)
#         if m:
#             modifier = m.group(1) # 'no' or 'normal'
#             obs = m.group(2)
#             loc = m.group(3)
#             concept_str = f"{modifier} {loc} {obs}" # Reordered f-string

#         # "calcified in the aorta" -> "calcified aorta" (general rule, keep)
#         m = re.match(r"^(.+?) in the (.+)$", concept_str)
#         if m:
#             obs, loc = m.group(1), m.group(2)
#             concept_str = f"{obs} {loc}"

#         concept_str = re.sub(r"\s+", " ", concept_str).strip()

#         # Rule for "no lungs" -> "no"
#         if concept_str == "no lungs":
#             concept_str = "no"

#         # Rule for removing 'lungs' if it appears alone after a descriptive term (e.g., 'normal lungs' becomes 'normal')
#         if concept_str.endswith(" lungs") and len(concept_str.split()) > 1:
#             last_word_before_lungs = concept_str.split()[-2]
#             if last_word_before_lungs in ["normal", "clear", "mild", "diffuse", "bilateral", "patchy", "both", "patent", "open"]:
#                 concept_str = concept_str.rsplit(' ', 1)[0] # Remove " lungs"

#         # Refined single-word leftover dropping
#         # Only drop if the single word is unlikely to be a meaningful concept on its own.
#         meaningful_single_words = ["normal", "patent", "open", "clear", "present", "absent", "pneumothorax", "effusion", "consolidation", "atelectasis", "mass", "nodule", "lesion", "fibrosis", "emphysema", "infiltrates", "thickening", "dilatation", "abnormality", "spleen", "trachea", "bronchi", "heart", "aorta", "esophagus", "mediastinum", "axilla", "carina", "subcarinal", "adrenal", "liver", "bone", "normal"]
#         if len(concept_str.split()) == 1 and concept_str not in meaningful_single_words:
#             concept_str = ""

#         if concept_str: # Only add non-empty concepts
#             processed_concepts.append(concept_str)

#     return processed_concepts

# gt_norm= []
# for concept_list in gt_concepts:
#     report_concepts = []
#     for concept in concept_list:
#         report_concepts.extend(canonicalize_concept(concept))
#     gt_norm.append([x for x in report_concepts if x])

# pred_norm= []
# for concept_list in pred_concepts:
#     report_concepts = []
#     for concept_str_raw in concept_list:
#         report_concepts.extend(canonicalize_concept(concept_str_raw))
#     pred_norm.append([x for x in report_concepts if x])

In [50]:
# def canonicalize_concept(s: str,is_pred: bool = False) -> list[str]:
#     s = str(s).strip().lower()
#     s = re.sub(r"\s+", " ", s)
#     s = s.rstrip(".")

#     # Split concepts by comma for adjective propagation
#     split_concepts = [c.strip() for c in s.split(',')] if is_pred else [s]
#     processed_concepts = []

#     for concept_str in split_concepts:
#         # Rule 1: Propagate leading adjectives (e.g., 'normal') across comma-separated concepts
#         # This logic needs to be careful not to apply to single concepts, only to comma-separated ones
#         if is_pred and len(split_concepts) > 1:
#             # Get the leading adjective from the *first* part of the comma-separated string
#             first_part_raw = split_concepts[0]
#             match_leading_adj = re.match(r"^(\w+)", first_part_raw)
#             leading_adjective = match_leading_adj.group(1) if match_leading_adj else ""

#             # Apply adjective to current concept_str if it doesn't already start with an adjective or "no"
#             current_concept_first_word = concept_str.split(' ')[0]
#             if leading_adjective and \
#                current_concept_first_word not in ["normal", "mild", "moderate", "severe", "slight", "minimal", "increased", "decreased", "diffuse", "bilateral", "patchy", "clear", "enlarged", "stable", "unchanged", "intact", "no"] and \
#                concept_str != first_part_raw:
#                 concept_str = f"{leading_adjective} {concept_str}"

#         # Canonicalization rules from original function
#         concept_str = concept_str.replace("top - normal", "normal")
#         concept_str = concept_str.replace("within normal limits", "normal")
#         concept_str = concept_str.replace("unremarkable", "normal")
#         # 1. Expanded Replacements: Add 'calibration' replacement with 'normal'.
#         concept_str = concept_str.replace("calibration", "normal")

#         # 2. Improved Reordering: Reorder "no/normal <observation> in the <location>" to "no/normal <location> <observation>"
#         m = re.match(r"^(no|normal) (.+?) in the (.+)$", concept_str)
#         if m:
#             modifier = m.group(1)
#             obs = m.group(2)
#             loc = m.group(3)
#             concept_str = f"{modifier} {loc} {obs}"
#         else:
#             # 3. Update the regex for general "X in the Y" patterns to reorder as "Y X"
#             m = re.match(r"^(.+?) in the (.+)$", concept_str)
#             if m:
#                 obs, loc = m.group(1), m.group(2)
#                 concept_str = f"{loc} {obs}"

#         concept_str = re.sub(r"\s+", " ", concept_str).strip()

#         # Rule for "no lungs" -> "no"
#         if concept_str == "no lungs":
#             concept_str = "no"

#         # 4. Refined 'Lungs' Handling: Expand the list of descriptive terms that allow 'lungs' to be removed
#         # Only remove 'lungs' if it appears alone after a descriptive term.
#         if concept_str.endswith(" lungs") and len(concept_str.split()) > 1:
#             last_word_before_lungs = concept_str.split()[-2]
#             if last_word_before_lungs in ["normal", "clear", "mild", "diffuse", "bilateral", "patchy", "both", "patent", "open", "few", "multiple", "several", "subcentimeter", "widespread"]:
#                 concept_str = concept_str.rsplit(' ', 1)[0] # Remove " lungs"

#         # Rule 2: Remove redundant 'lungs' qualifier if it directly follows 'no' (original rule, keep)
#         if is_pred and concept_str.startswith("no lungs "):
#             concept_str = concept_str.replace("no lungs ", "no ", 1)

#         # 5. Expanded Meaningful Single Words
#         # Only drop if the single word is unlikely to be a meaningful concept on its own.
#         meaningful_single_words = [
#             "normal", "patent", "open", "clear", "present", "absent", "pneumothorax", "effusion", "consolidation",
#             "atelectasis", "mass", "nodule", "lesion", "fibrosis", "emphysema", "infiltrates", "thickening",
#             "dilatation", "abnormality", "spleen", "trachea", "bronchi", "heart", "aorta", "esophagus",
#             "mediastinum", "axilla", "carina", "subcarinal", "adrenal", "liver", "bone",
#             "calibration", "diameter", "widths", "contour", "size", "alignments", "densities", "preserved",
#             "suboptimal", "triangular", "fashion", "remnant", "suggestive", "findings", "stable", "other",
#             "pneumonic", "infiltration", "occlusive", "pathology", "involvement", "abnormality", "calcified",
#             "cystic", "tubular", "bronchiectasis", "bronchopneumonic", "infiltrates", "air", "trapping",
#             "secretion", "lumen", "catheter", "vascular", "structures", "vessels", "heart", "aorta", "esophagus",
#             "mediastinum", "hilum", "axilla", "supraclavicular", "fossa", "carina", "subcarinal", "adrenal",
#             "kidney", "liver", "spleen", "pancreas", "gallbladder", "bowel", "stomach", "pelvis", "abdomen",
#             "chest", "thoracic", "cervical", "lumbar", "sacral", "bone", "vertebral", "disc", "foramina", "ribs",
#             "sternum", "clavicle", "scapula", "humerus", "femur", "tibia", "fibula", "patella", "phalanges",
#             "skull", "brain", "sinuses", "orbit", "neck", "thyroid", "parathyroid", "glands", "nodular", "lesions",
#             "mass", "cyst", "opacities", "ground", "glass", "consolidation", "atelectasis", "effusion",
#             "pneumothorax", "emphysema", "fibrosis", "scarring", "thickening", "dilatation", "rupture", "fracture",
#             "edema", "hemorrhage", "infarction", "ischemia", "embolism", "thrombus", "stricture", "stenosis",
#             "obstruction", "perfusion", "ventilation", "drainage", "tube", "device", "hardware", "clip", "stent",
#             "shunt", "catheter", "drain", "electrode", "wire", "pacemaker", "defibrillator", "implant", "prosthesis",
#             "suture", "staple", "screw", "plate", "rod", "nail", "fixation", "graft", "flap", "transplant",
#             "foreign", "body", "infection", "inflammation", "abscess", "tumor", "malignancy", "metastasis",
#             "benign", "hyperplasia", "atrophy", "dystrophy", "sclerosis", "degeneration", "necrosis", "granuloma",
#             "calcification", "ossification", "pneumonia", "bronchiolitis", "pleuritis", "pericarditis",
#             "myocarditis", "endocarditis", "aortitis", "phlebitis", "lymphadenopathy", "hepatomegaly",
#             "splenomegaly", "nephromegaly", "cholelithiasis", "nephrolithiasis", "urolithiasis", "ascites",
#             "hernia", "diverticulitis", "colitis", "appendicitis", "pancreatitis", "cholecystitis", "hepatitis",
#             "cirrhosis", "cholecystectomy", "nephrectomy", "splenectomy", "appendectomy", "gastrectomy",
#             "colectomy", "thyroidectomy", "mastectomy", "lymphadenectomy", "biopsy", "resection", "ablation",
#             "embolization", "ligation", "reduction", "repair", "drainage", "aspiration", "puncture", "injection",
#             "infusion", "therapy", "treatment", "medication", "radiation", "chemotherapy", "surgery", "intervention",
#             "procedure", "examination", "study", "scan", "xray", "ct", "mri", "ultrasound", "angiography",
#             "fluoroscopy", "nuclear", "medicine", "endoscopy", "bronchoscopy", "colonoscopy", "gastroscopy",
#             "cystoscopy", "arthroscopy", "laparoscopy", "thoracoscopy", "mediastinoscopy", "hysteroscopy",
#             "colposcopy", "ophthalmoscopy", "otoscopy", "laryngoscopy", "pharyngoscopy", "rhinoscopy", "sinuscopy",
#             "colonography", "sigmoidoscopy", "anoscopy", "proctoscopy", "rectoscopy", "ureteroscopy", "nephroscopy",
#             "pyeloscopy", "cholangioscopy", "pancreatoscopy", "duodenoscopy", "jejunoscopy", "ileoscopy", "cecum",
#             "appendix", "colon", "rectum", "anus", "small", "intestine", "duodenum", "jejunum", "ileum", "stomach",
#             "esophagus", "pharynx", "larynx", "trachea", "bronchi", "lungs", "pleura", "diaphragm", "pericardium",
#             "myocardium", "endocardium", "aorta", "arteries", "veins", "capillaries", "lymph", "nodes", "ducts",
#             "glands", "thymus", "thyroid", "parathyroid", "adrenal", "pancreas", "spleen", "liver", "gallbladder",
#             "kidneys", "ureters", "bladder", "urethra", "gonads", "uterus", "fallopian", "tubes", "ovaries",
#             "vagina", "penis", "testes", "prostate", "epididymis", "vas", "deferens", "seminal", "vesicles",
#             "mammary", "glands", "breast", "skin", "subcutaneous", "tissue", "fascia", "muscle", "tendon",
#             "ligament", "cartilage", "bone", "marrow", "joint", "capsule", "synovium", "bursa", "meniscus", "disc",
#             "spine", "vertebrae", "skull", "brain", "cerebrum", "cerebellum", "brainstem", "ventricles", "meninges",
#             "nerves", "spinal", "cord", "plexus", "ganglia", "eyes", "orbits", "lids", "conjunctiva", "cornea",
#             "lens", "retina", "optic", "nerve", "ears", "outer", "middle", "inner", "nose", "sinuses", "mouth",
#             "tongue", "salivary", "glands", "teeth", "gums", "palate", "tonsils", "pharynx", "larynx", "vocal",
#             "cords", "epiglottis", "thyroid", "cartilage", "cricoid", "artery", "vein", "nerve", "lymphatic",
#             "vessel", "capillary", "gland", "duct", "organ", "tissue", "cell", "membrane", "structure", "component",
#             "body", "region", "area", "site", "location", "part", "portion", "segment", "lobe", "fissure", "sulcus",
#             "gyrus", "nucleus", "tract", "bundle", "layer", "sheath", "space", "cavity", "lumen", "wall", "margin",
#             "border", "contour", "shape", "size", "density", "attenuation", "signal", "intensity", "pattern",
#             "texture", "enhancement", "perfusion", "flow", "motion", "function", "activity", "metabolism",
#             "viability", "integrity", "continuity", "caliber", "width", "height", "length", "depth", "volume",
#             "mass", "weight", "age", "sex", "history", "clinical", "symptoms", "signs", "diagnosis", "etiology",
#             "pathology", "prognosis", "treatment", "response", "complication", "sequela", "finding", "observation",
#             "impression", "conclusion", "recommendation", "followup", "comparison", "technique", "contrast", "dose",
#             "protocol", "series", "image", "view", "slice", "plane", "orientation", "artifact", "noise", "quality",
#             "limitation", "variant", "normal", "abnormal", "concerning", "suspicious", "malignant", "benign",
#             "inflammatory", "infectious", "traumatic", "vascular", "ischemic", "hemorrhagic", "confiltrative",
#             "obstructive", "dilated", "stenotic", "ruptured", "fractured", "displaced", "rotated", "migrated",
#             "perforated", "eroded", "ulcerated", "calcified", "ossified", "necrotic", "atrophic", "hypertrophic",
#             "hyperplastic", "dystrophic", "sclerotic", "degenerative", "edematous", "congested", "pneumonitic",
#             "pleuritic", "pericarditic", "myocarditic", "endocarditic", "aortitic", "phlebitic",
#             "lymphadenopathic", "hepatomegalic", "splenomegalic", "nephromegalic", "cholelithic", "nephrolithic",
#             "urolithic", "ascitic", "herniated", "diverticulitic", "colitic", "appendicitic", "pancreatitis",
#             "cholecystitis", "hepatitis", "cirrhosis"
#         ]
#         meaningful_single_words = list(set(meaningful_single_words)) # Remove duplicates

#         if len(concept_str.split()) == 1 and concept_str not in meaningful_single_words:
#             concept_str = ""

#         if concept_str: # Only add non-empty concepts
#             processed_concepts.append(concept_str)

#     return processed_concepts

# gt_norm= []
# for concept_list in gt_concepts:
#     report_concepts = []
#     for concept in concept_list:
#         report_concepts.extend(canonicalize_concept(concept, is_pred=False))
#     gt_norm.append([x for x in report_concepts if x])

# pred_norm= []
# for concept_list in pred_concepts:
#     report_concepts = []
#     for concept_str_raw in concept_list:
#         report_concepts.extend(canonicalize_concept(concept_str_raw, is_pred=True))
#     pred_norm.append([x for x in report_concepts if x])

In [51]:
def canonicalize_concept(s: str) -> list[str]:
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.rstrip(".")

    # Split concepts by comma for adjective propagation - now applied uniformly
    split_concepts = [c.strip() for c in s.split(',')]
    processed_concepts = []

    # Capture initial adjective for propagation, if available in the first part
    initial_adjective = ""
    if split_concepts:
        match_initial_adj = re.match(r"^(\w+)", split_concepts[0])
        if match_initial_adj:
            initial_adjective = match_initial_adj.group(1)

    for i, concept_str_raw in enumerate(split_concepts):
        concept_str = concept_str_raw

        # Adjective Propagation for subsequent parts
        if i > 0 and initial_adjective and \
           not re.match(r"^(no|normal|mild|moderate|severe|slight|minimal|increased|decreased|diffuse|bilateral|patchy|clear|enlarged|stable|unchanged|intact|present|absent)", concept_str.split(' ')[0]):
            concept_str = f"{initial_adjective} {concept_str}"

        # Canonicalization rules
        concept_str = concept_str.replace("top - normal", "normal")
        concept_str = concept_str.replace("within normal limits", "normal")
        concept_str = concept_str.replace("unremarkable", "normal")

        # Reorder "no/normal <observation> in the <location>" to "no/normal <location> <observation>"
        m = re.match(r"^(no|normal) (.+?) in the (.+)$", concept_str)
        if m:
            modifier = m.group(1)
            obs = m.group(2)
            loc = m.group(3)
            concept_str = f"{modifier} {loc} {obs}"
        else:
            # Refined reordering for general "X in the Y" patterns to "Y X" (e.g., "effusion in the pleural" -> "pleural effusion")
            # This now prioritizes location before observation for better alignment with GT
            m = re.match(r"^(.+?) in the (.+)$", concept_str)
            if m:
                obs, loc = m.group(1), m.group(2)
                concept_str = f"{loc} {obs}"

        concept_str = re.sub(r"\s+", " ", concept_str).strip()

        # Rule for "no lungs" -> "no"
        if concept_str == "no lungs":
            concept_str = "no"

        # Rule for removing 'lungs' if it appears alone after a descriptive term
        if concept_str.endswith(" lungs") and len(concept_str.split()) > 1:
            last_word_before_lungs = concept_str.split()[-2]
            if last_word_before_lungs in ["normal", "clear", "mild", "diffuse", "bilateral", "patchy", "both", "patent", "open", "few", "multiple", "several", "subcentimeter", "widespread", "all"]:
                concept_str = concept_str.rsplit(' ', 1)[0] # Remove " lungs"

        # Refined single-word leftover dropping
        # Expanded meaningful_single_words list
        meaningful_single_words = [
            "normal", "patent", "open", "clear", "present", "absent", "pneumothorax", "effusion", "consolidation",
            "atelectasis", "mass", "nodule", "lesion", "fibrosis", "emphysema", "infiltrates", "thickening",
            "dilatation", "abnormality", "spleen", "trachea", "bronchi", "heart", "aorta", "esophagus",
            "mediastinum", "axilla", "carina", "subcarinal", "adrenal", "liver", "bone", "pneumonia",
            "infection", "vascular", "structures", "cysts", "scarring", "fluid", "collection", "size",
            "contour", "caliber", "diameter", "heights", "margins", "wall", "tubes", "catheter", "implant",
            "opacities", "ground-glass", "density", "findings", "progression", "regression", "stable", "changes"
        ]
        meaningful_single_words = list(set(meaningful_single_words)) # Remove duplicates

        if len(concept_str.split()) == 1 and concept_str not in meaningful_single_words:
            concept_str = ""

        if concept_str: # Only add non-empty concepts
            processed_concepts.append(concept_str)

    return processed_concepts

gt_norm= []
for concept_list in gt_concepts:
    report_concepts = []
    for concept in concept_list:
        report_concepts.extend(canonicalize_concept(concept))
    gt_norm.append([x for x in report_concepts if x])

pred_norm= []
for concept_list in pred_concepts:
    report_concepts = []
    for concept_str_raw in concept_list:
        report_concepts.extend(canonicalize_concept(concept_str_raw))
    pred_norm.append([x for x in report_concepts if x])

In [52]:
for i in (range(len(pred_concepts))):
  print(reports[i])
  print(i," :")
  print(gt_norm[i],'\n',pred_norm[i],'\n')


The trachea and both main bronchi are open. An air cyst is observed at the anterobasal level of the right lower lobe. Focal consolidative density is observed at the paramediastinal level of the right middle lobe. There are focal scarring changes at the posterobasal level of the left lung. A calcified nodule measuring 3 mm is observed at the laterobasal level. No bilateral pleural effusion or pneumothorax is detected. Mediastinal main vascular structures, heart contour, and size are normal. Thoracic aorta diameter is normal. No pericardial effusion or thickening is observed. Thoracic esophagus is normal in caliber without significant wall thickening. No enlarged lymph nodes are detected in prevascular, pre-paratracheal, subcarinal, or bilateral hilar-axillary regions. Bone structures within the field of view are normal. Vertebral body heights are preserved. A nodular formation compatible with an accessory spleen is observed adjacent to the spleen.
0  :
['patent air passages', 'nodular c

In [53]:
def similarity2(a: str, b: str) -> float:
    a_tokens = set(a.lower().split())
    b_tokens = set(b.lower().split())

    if not a_tokens and not b_tokens:
        return 1.0
    if not a_tokens or not b_tokens:
        return 0.0

    intersection = a_tokens & b_tokens
    union = a_tokens | b_tokens

    return (2*len(intersection)) / (len(a_tokens)+len(b_tokens))


In [54]:
def fuzzy_counts_one(gt_list, pred_list, threshold):
    gt = [g.strip().lower() for g in gt_list if g.strip()]
    pr = [p.strip().lower() for p in pred_list if p.strip()]

    used_gt = set()
    TP = 0

    for p in pr:
        best_j = None
        best_score = 0.0

        for j, g in enumerate(gt):
            if j in used_gt:
                continue

            score = similarity2(p, g)
            if score > best_score:
                best_score = score
                best_j = j

        if best_j is not None and best_score >= threshold:
            TP += 1
            used_gt.add(best_j)

    FP = len(pr) - TP
    FN = len(gt) - TP

    return TP, FP, FN


In [55]:
def fuzzy_prf(gt_norm, pred_norm, threshold):
    TP = FP = FN = 0

    for gt, pr in zip(gt_norm, pred_norm):
        t, f, n = fuzzy_counts_one(gt, pr, threshold)
        TP += t
        FP += f
        FN += n

    precision = TP / (TP + FP) if TP + FP else 0
    recall    = TP / (TP + FN) if TP + FN else 0
    f1        = (2 * precision * recall / (precision + recall)) if precision + recall else 0

    return precision, recall, f1, (TP, FP, FN)


In [56]:

for th in [0.7,0.75, 0.8, 0.85,0.9]:
    P, R, F1, counts = fuzzy_prf(gt_norm, pred_norm, threshold=th)
    print(th, P, R, F1)

0.7 0.14673913043478262 0.20149253731343283 0.169811320754717
0.75 0.14130434782608695 0.19402985074626866 0.16352201257861634
0.8 0.13043478260869565 0.1791044776119403 0.1509433962264151
0.85 0.10326086956521739 0.1417910447761194 0.11949685534591196
0.9 0.08152173913043478 0.11194029850746269 0.09433962264150943
